### Reverse Geocoding
&nbsp;
- ITRF2000 좌표계로 표시된 좌표를 위도, 경도 값으로 변환합니다.
    - pip install pyproj
    
- 위도, 경도 값은 카카오 Open API를 사용해 실제 주소로 변환합니다.

In [13]:
from pyproj import Proj, transform
import json

def change_lat_lng(path_x, path_y):
    
    itrf_x, itrf_y = path_x, path_y
    
    proj_UTMK = Proj(init='epsg:5178') #기존 좌표계: ITRF2000
    proj_WGS84 = Proj(init='epsg:4326') #변환할 좌표계
    
    x, y = transform(proj_UTMK, proj_WGS84, itrf_x, itrf_y)
    latlng = (x, y)
    
    return latlng

In [14]:
latlng_df = pd.DataFrame()
raw_df = pd.read_csv("./raw_datas/jeju_local_latlng.csv", encoding='utf-8')

In [15]:
def get_transform_xy():
    transform_xy = []
    latlng_df = raw_df.loc[::, ["POINT_X", "POINT_Y"]]
    
    for i in range(50000):
        raw_x = latlng_df['POINT_X'][i]
        raw_y = latlng_df['POINT_Y'][i]

        res = change_lat_lng(raw_x, raw_y)
        transform_xy.append(res)

    return transform_xy

In [16]:
def get_search_url(req_x, req_y):
    
    base = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"     
    header_phrase = "KakaoAK"
    api_key = " 8d955ef0e4d59d215f48c2998b8ff5d9"
    headers = {"Authorization": header_phrase + api_key}
    
    req_lat = "?x=" + str(req_x)
    req_lng = "&y=" + str(req_y)
    
    url = base + req_lat + req_lng
    req_set = (url, headers)
    
    return req_set

In [17]:
def get_result(url, header):
    
    req = requests.get(url, headers=header)
    res = json.loads(req.text)
 
    return res

In [18]:
def reverse_geocoding(req_x, req_y):

    req_set = get_search_url(req_x, req_y)
    res_api = get_result(req_set[0], req_set[1])
    
    return res_api

In [19]:
def json_parsing(transform_xy):
    
    api_addrs = []

    for i in range(50000):
        x = str(transform_xy[i][0])
        y = str(transform_xy[i][1])

        res_api = reverse_geocoding(x, y)
        json_datas = res_api.items()

        for key, value in json_datas:
            if type(value) == list:
                address = value[0]['address_name']
                
        api_addrs.append(address)
    
    print(api_addrs)
    
    return api_addrs 

In [ ]:
# start reverse geocoding
transform_xy = get_transform_xy()
api_addrs = json_parsing(transform_xy)

In [9]:
api_addrs

['제주특별자치도 제주시 한경면 고산리',
 '제주특별자치도 제주시 한경면 고산리',
 '제주특별자치도 제주시 한경면 고산리',
 '제주특별자치도 제주시 한경면 고산리',
 '제주특별자치도 제주시 한경면 고산리',
 '제주특별자치도 제주시 한경면 고산리',
 '제주특별자치도 제주시 한경면 두모리',
 '제주특별자치도 제주시 한경면 두모리',
 '제주특별자치도 제주시 한경면 두모리',
 '제주특별자치도 서귀포시 대정읍 영락리',
 '제주특별자치도 제주시 한경면 판포리',
 '제주특별자치도 제주시 한림읍 월령리',
 '제주특별자치도 제주시 한림읍 금능리',
 '제주특별자치도 제주시 한림읍 금능리',
 '제주특별자치도 제주시 한림읍 금능리',
 '제주특별자치도 제주시 한림읍 금능리',
 '제주특별자치도 제주시 한림읍 금능리',
 '제주특별자치도 제주시 한림읍 금능리',
 '제주특별자치도 제주시 한림읍 금능리',
 '제주특별자치도 제주시 한림읍 협재리',
 '제주특별자치도 제주시 한림읍 협재리',
 '제주특별자치도 제주시 한림읍 협재리',
 '제주특별자치도 제주시 한림읍 협재리',
 '제주특별자치도 제주시 한림읍 협재리',
 '제주특별자치도 제주시 한림읍 협재리',
 '제주특별자치도 제주시 한림읍 협재리',
 '제주특별자치도 제주시 한림읍 협재리',
 '제주특별자치도 제주시 한림읍 협재리',
 '제주특별자치도 제주시 한림읍 협재리',
 '제주특별자치도 제주시 한림읍 협재리',
 '제주특별자치도 제주시 한림읍 협재리',
 '제주특별자치도 서귀포시 대정읍 동일리',
 '제주특별자치도 제주시 한림읍 협재리',
 '제주특별자치도 제주시 한림읍 협재리',
 '제주특별자치도 서귀포시 대정읍 동일리',
 '제주특별자치도 제주시 한림읍 협재리',
 '제주특별자치도 제주시 한림읍 협재리',
 '제주특별자치도 서귀포시 대정읍 하모리',
 '제주특별자치도 서귀포시 대정읍 하모리',
 '제주특별자치도 서귀포시 대정읍 하모리',
 '제주특별자치도 제주시 한림읍 협재리',
 '제주특별자치도 

----

In [10]:
addr_df = pd.DataFrame(columns=['address'])

for i in range(len(api_addrs)):
    addr_df.loc[i] = api_addrs[i]

addr_df.head()

fulladdr_df = pd.concat([raw_df, addr_df], axis=1)
fulladdr_df # full address dataframe

,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,address
0,1,202005,제주특별자치도,제주시,영세,일반한식,00시,363000,66500,10,2,877005.9834,1479766.073,제주특별자치도 제주시 한경면 고산리
1,2,202005,제주특별자치도,제주시,영세,단란주점,00시,1180000,0,3,0,877005.7447,1479816.049,제주특별자치도 제주시 한경면 고산리
2,3,202005,제주특별자치도,제주시,중소1,편의점,00시,157670,6850,20,2,877056.6756,1479616.384,제주특별자치도 제주시 한경면 고산리
3,4,202005,제주특별자치도,제주시,영세,편의점,00시,46600,0,2,0,877055.9593,1479766.312,제주특별자치도 제주시 한경면 고산리
4,5,202005,제주특별자치도,제주시,영세,주점,00시,66000,0,2,0,877055.4817,1479866.263,제주특별자치도 제주시 한경면 고산리
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273178,273179,202005,제주특별자치도,제주시,중소,침구수예점,x시,17000,0,1,0,949616.6171,1502504.547,NaN
273179,273180,202005,제주특별자치도,제주시,중소1,서양음식,x시,12000,0,1,0,949665.1566,1502804.667,NaN
273180,273181,202005,제주특별자치도,제주시,영세,편의점,x시,4800,0,1,0,950266.5976,1502457.688,NaN
273181,273182,202005,제주특별자치도,제주시,중소,제과점,x시,25000,0,2,0,950524.6587,1500759.566,NaN
